# DeepChem

- [Installation](https://github.com/deepchem/deepchem#installation)
- [Tutorial](https://deepchem.readthedocs.io/en/latest/get_started/tutorials.html)
- [Sample Notebooks](https://github.com/deepchem/deepchem/tree/master/examples/tutorials) 

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import deepchem as dc

from sklearn.metrics import r2_score, mean_squared_error

# Parameters , Settings

In [21]:
data_filepath = './data/ESOL.csv'   

model_dir = './result'   # folder to save fitted model
save_hyperparms = './result/Hyperparameter'   # folder to save models during hyperparameter optimization

n_tasks = 1   # No. of tasks (No. of dependent variables)

In [22]:
# Initialze the metrics
# https://deepchem.readthedocs.io/en/latest/api_reference/metrics.html
metric_r2 = dc.metrics.Metric(dc.metrics.r2_score)
metric_mse = dc.metrics.Metric(dc.metrics.mean_squared_error)
metrics = [metric_r2, metric_mse]

# Load Data 

In [23]:
data = pd.read_csv(data_filepath)   
data.shape

(1128, 2)

In [24]:
smiles = data['smiles']   # should be 1D 
y = data['measured log solubility in mols per litre']   # can be 1D or 2D

# Make Dataset for DeepChem

In [6]:
''' ConvMolFeaturizer '''
# Duvenaud graph convolutions ; Can be used with Keras models
# https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html#convmolfeaturizer

featurizer = dc.feat.ConvMolFeaturizer()
features = featurizer.featurize(smiles)   # numpy array, it returns a graph object for every molecule
type(features[0])   # deepchem.feat.mol_graphs.ConvMol

deepchem.feat.mol_graphs.ConvMol

In [7]:
# "NumpyDataset" class stores datasets in memory. This works fine for smaller datasets, but is less convenient for larger datasets. 
# For large datasets, use "DiskDataset" which writes dataset to a folder. 
# https://deepchem.readthedocs.io/en/latest/api_reference/data.html
# https://github.com/deepchem/deepchem/blob/master/examples/tutorials/02_Working_With_Datasets.ipynb
dataset = dc.data.NumpyDataset(X=features, y=np.array(y))
print(dataset)

<NumpyDataset X.shape: (1128,), y.shape: (1128,), w.shape: (1128,), task_names: [0]>


# Split Dataset

In [8]:
# "random splitting" sometimes overestimates model’s performance, especially for small data or imbalance data. 
# The dc.splits provides more methods and algorithms to evaluate the model’s performance appropriately, like cross validation or splitting using molecular scaffolds.
# https://deepchem.readthedocs.io/en/latest/api_reference/splitters.html

splitter = dc.splits.RandomSplitter()

train_dataset, test_dataset = splitter.train_test_split(dataset=dataset, frac_train=0.8, seed=0)
print(test_dataset)

<NumpyDataset X.shape: (226,), y.shape: (226,), w.shape: (226,), ids: [148 1014 1021 ... 835 559 684], task_names: [0]>


# Model , Train

In [10]:
# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#graphconvmodel
# This Class uses Keras models
model = dc.models.GraphConvModel(n_tasks=n_tasks,              # No. of tasks 
                                 graph_conv_layers=[64, 64],   # Width of channels for the Graph Convolution Layers
                                 dense_layer_size=128,         # Width of channels for Atom Level Dense Layer before GraphPool
                                 dropout=0.2,                  # Dropout probablity to use for each layer. The length of this list should equal len(graph_conv_layers)+1 (one value for each convolution layer, and one for the dense layer). Alternatively this may be a single value instead of a list, in which case the same value is used for every layer.
                                 mode='regression',            # Either “classification” or “regression”
                                 batch_size=100,               # Batch size for training and evaluating
                                 model_dir=model_dir,          # Directory on disk where the model will be stored. If this is None, a temporary directory is created.
                                 learning_rate=0.001
                                 )

# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#kerasmodel
loss_avg = model.fit(train_dataset, nb_epoch=50)

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(353,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(353, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1314,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(349,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(349, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1264,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

In [11]:
model.model.summary()   # must be after "model.fit"

Model: "private__graph_conv_keras_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
graph_conv (GraphConv)       multiple                  102144    
_________________________________________________________________
graph_conv_1 (GraphConv)     multiple                  87360     
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
batch_normalization_1 (Batch multiple                  256       
_________________________________________________________________
batch_normalization_2 (Batch multiple                  512       
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multip

In [12]:
# Evaluate the model
print("Training set score:", model.evaluate(train_dataset, metrics))
print("Test set score:", model.evaluate(test_dataset, metrics))

Training set score: {'r2_score': 0.7225232445921097, 'mean_squared_error': 1.224374317157278}
Test set score: {'r2_score': 0.6660339328231306, 'mean_squared_error': 1.4380285752099478}


In [13]:
model.get_checkpoints()   # location of fitted model

['./result/ckpt-1']

### Prediction

In [15]:
# input to the "predict" is "dataset"
y_pred = model.predict(test_dataset)   # numpy array [n, 1]

In [18]:
# Since we have predicted values, we can use scikit-learn methods to calculate metrics as usual.
y_test = test_dataset.y

print("Test dataset (R2) : ", r2_score(y_test, y_pred))
print("Test dataset (RMSE) : ", np.sqrt(mean_squared_error(y_test, y_pred)))

Test dataset (R2) :  0.6660339328231306
Test dataset (RMSE) :  1.1991782916689027


# Reload Saved Model

In [19]:
del model

# Initialize a model with the same parameters when you defined model for the 1st time (before training)
model = dc.models.GraphConvModel(n_tasks=n_tasks,              # No. of tasks 
                                 graph_conv_layers=[64, 64],   # Width of channels for the Graph Convolution Layers
                                 dense_layer_size=128,         # Width of channels for Atom Level Dense Layer before GraphPool
                                 dropout=0.2,                  # Dropout probablity to use for each layer. The length of this list should equal len(graph_conv_layers)+1 (one value for each convolution layer, and one for the dense layer). Alternatively this may be a single value instead of a list, in which case the same value is used for every layer.
                                 mode='regression',            # Either “classification” or “regression”
                                 batch_size=100,               # Batch size for training and evaluating
                                 model_dir=model_dir,          # Directory on disk where the model will be stored. If this is None, a temporary directory is created.
                                 learning_rate=0.001
                                 )

# fitted model is saved in "model_dir" and we can restore it.
model.restore()

In [21]:
# Test prediction
# input to the "predict" is "dataset"
y_pred = model.predict(test_dataset)   # numpy array [n, 1]

print("Test dataset (R2) : ", r2_score(y_test, y_pred))
print("Test dataset (RMSE) : ", np.sqrt(mean_squared_error(y_test, y_pred)))

Test dataset (R2) :  0.6660339328231306
Test dataset (RMSE) :  1.1991782916689027


# Uncertainty

Uncertainties are estimates, not rigorous measurements. Most of them are pretty good, but you should not put too much confidence in any single value.

It involves repeating the prediction many times with different dropout masks.

If a model supports uncertainty, it MUST use dropout on every layer, and dropout most be enabled during uncertainty prediction. Otherwise, the uncertainties it computes will be inaccurate

https://github.com/deepchem/deepchem/blob/master/examples/tutorials/25_Uncertainty_In_Deep_Learning.ipynb

In [22]:
del model

# Initialize model with parameter "uncertainty=True"
model = dc.models.GraphConvModel(n_tasks=n_tasks,              # No. of tasks 
                                 graph_conv_layers=[64, 64],   # Width of channels for the Graph Convolution Layers
                                 dense_layer_size=128,         # Width of channels for Atom Level Dense Layer before GraphPool
                                 dropout=0.2,                  # Dropout probablity to use for each layer. The length of this list should equal len(graph_conv_layers)+1 (one value for each convolution layer, and one for the dense layer). Alternatively this may be a single value instead of a list, in which case the same value is used for every layer.
                                 mode='regression',            # Either “classification” or “regression”
                                 batch_size=100,               # Batch size for training and evaluating
                                 model_dir=model_dir,          # Directory on disk where the model will be stored. If this is None, a temporary directory is created.
                                 learning_rate=0.001,
                                 uncertainty=True
                                 )

loss_avg = model.fit(train_dataset, nb_epoch=50)

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_14:0", shape=(342,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_13:0", shape=(342, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_17:0", shape=(1212,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_14:0", shape=(366,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_13:0", shape=(366, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_pool_5/Reshape_17:0", shape=(1260,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_conv_5/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_conv_5/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_conv_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_2/graph_conv_5/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

In [24]:
# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#deepchem.models.KerasModel.predict_uncertainty
y_pred, y_std = model.predict_uncertainty(test_dataset)   # numpy array
y_std[:5]

array([[2.187775 ],
       [2.1478465],
       [1.9598346],
       [1.6264415],
       [1.8399079]], dtype=float32)

# Hyperparameter Optimization (Grid Search)


In [11]:
# First, we split dataset again into train / validation / test. 
# Hyperparameter Optimization requires validation dataset. Best model is selected based on the metric of validation dataset. 
splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, 
            frac_train=0.7, frac_valid=0.15, frac_test=0.15)

In [26]:
# Values in "params_dict" must be List
params_dict = {
    'n_tasks': [n_tasks],
    'mode': ['regression'],   
    'graph_conv_layers': [[64], [64, 64]],
    'dropout': [0.2, 0.5]
}

In [27]:
# Grid Search
# https://deepchem.readthedocs.io/en/latest/api_reference/hyper.html#grid-hyperparameter-optimization
optimizer = dc.hyper.GridHyperparamOpt(dc.models.GraphConvModel)

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(
            params_dict,       # dictionary that maps hyperparameter names (strings) to lists of possible parameter values
            train_dataset, 
            valid_dataset,     # dataset used for validation (optimization on valid scores) 
            metric_r2,         # metric used for evaluation
            nb_epoch=50,
            use_max=True,      # If True, return the model with the highest score. Else return model with the minimum score.
            logdir=save_hyperparms   # The directory in which to store created models. If not set, will use a temporary directory.
            )
# all_results contains value of metric for validation dataset

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_6/Reshape_14:0", shape=(362,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_6/Reshape_13:0", shape=(362, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_6/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_6/Reshape_17:0", shape=(1096,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_7/Reshape_14:0", shape=(358,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_7/Reshape_13:0", shape=(358, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_7/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_4/graph_pool_7/Reshape_17:0", shape=(1296,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_9/Reshape_14:0", shape=(354,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_9/Reshape_13:0", shape=(354, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_9/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_pool_9/Reshape_17:0", shape=(1254,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_conv_9/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_conv_9/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_conv_9/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_5/graph_conv_9/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_11/Reshape_14:0", shape=(370,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_11/Reshape_13:0", shape=(370, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_11/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_pool_11/Reshape_17:0", shape=(1214,), dtype=int32), values=Tensor("gradient_tape/private__graph_co

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_conv_11/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_conv_11/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_conv_11/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_6/graph_conv_11/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_co

In [28]:
best_hyperparams

(1, 'regression', [64], 0.2)

In [32]:
all_results

{'_dropouts_0.200000_graph_conv_layers[64]_moderegression_n_tasks_1': 0.746424775042789,
 '_dropouts_0.500000_graph_conv_layers[64]_moderegression_n_tasks_1': 0.7007858304236246,
 '_dropouts_0.200000_graph_conv_layers[64, 64]_moderegression_n_tasks_1': 0.6978143017856612,
 '_dropouts_0.500000_graph_conv_layers[64, 64]_moderegression_n_tasks_1': 0.7285161522316366}

In [29]:
best_model.get_checkpoints()   # filename of best model saved in the folder "save_hyperparms"

['./result/Hyperparameter/0/ckpt-1']

### Load saved best model

In [33]:
best_model_checkpoints = best_model.get_checkpoints()[0]
best_model_folder = os.path.dirname(best_model_checkpoints)   # name of folder where best model is saved

del model

# Initialize model with best hyperparameters
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', graph_conv_layers=[64], 
                                 dropout=0.2, model_dir=best_model_folder)

model.restore()

In [34]:
print("Training set score:", model.evaluate(train_dataset, metric_r2))
print("Validation set score:", model.evaluate(valid_dataset, metric_r2))
print("Test set score:", model.evaluate(test_dataset, metric_r2))

Training set score: {'r2_score': 0.9459974181683233}
Validation set score: {'r2_score': 0.746424775042789}
Test set score: {'r2_score': 0.7605521890903217}


# Hyperparameter Optimization (Gaussian Process)

#### Gaussian Process Global Optimization (GPGO)

https://deepchem.readthedocs.io/en/latest/api_reference/hyper.html#gaussian-process-hyperparameter-optimization

https://github.com/deepchem/deepchem/tree/master/examples/hyperparam_opt

It uses pyGPGO to optimize hyperparameters : pip install pyGPGO


"params_dict" has a different semantics than for GridHyperparamOpt. param_dict[key] must be an int/float, which is used as the center of a search with radius "search_range" since pyGPGO can only optimize numerical hyperparameters.

"search_range" specifies the range of parameter values to search for. If search_range is an int/float, it is used as the global search range for parameters. This creates a search problem on the following space: 

optimization on (initial value / search_range, initial value * search_range)


In [9]:
# Values in "params_dict" are initial values (center of the search space) for the corresponding key
params_dict = {'dropout': 0.3, 'learning_rate' : 0.001}

# Radius of search space , must have the same Key as "params_dict"
search_range = {'dropout': 2, 'learning_rate' : 5}   

In [12]:
# We don't want to optimize all parameters. Some parameters like 'mode' are fixed and defined below:
optimizer = dc.hyper.GaussianProcessHyperparamOpt(
    lambda **p: dc.models.GraphConvModel(n_tasks=n_tasks, mode="regression", **p))

best_model, best_params, all_results = optimizer.hyperparam_search(
            params_dict,       # Dictionary that maps hyperparameter names (strings) to the possible parameter values 
            train_dataset, 
            valid_dataset,     # Dataset used for validation (optimization on valid scores 
            metric_r2,         # Metric used for evaluation
            nb_epoch=50,
            use_max=True,      # If True, return the model with the highest score. Else return model with the minimum score.
            logdir=save_hyperparms,   # Directory in which to store created models. If not set, will use a temporary directory.    
            max_iter=10,        # No. of optimization trials
            search_range=search_range
            )

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(395,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(395, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1154,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(378,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(378, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1114,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_conv_1/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_mod

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Reshape_14:0", shape=(369,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Reshape_13:0", shape=(369, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_1/graph_pool_3/Reshape_17:0", shape=(1082,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

...

9      	 [0.15387001 0.00366433]. 	  0.5786634529026986 	 0.6009713994268596


/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_14:0", shape=(368,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_13:0", shape=(368, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_17:0", shape=(980,), dtype=int32), values=Tensor("gradient_tape/private__graph

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_14:0", shape=(337,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_13:0", shape=(337, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_pool_25/Reshape_17:0", shape=(1156,), dtype=int32), values=Tensor("gradient_tape/private__grap

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_conv_25/Reshape_11:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_conv_25/Reshape_10:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_conv_25/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_12/graph_conv_25/Reshape_13:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/private__grap

10     	 [0.20384127 0.0038913 ]. 	  0.5100267062642383 	 0.6009713994268596


In [13]:
best_params

{'dropout': 0.15, 'learning_rate': 0.005}

In [14]:
all_results

{'_dropout_0.257594_learning_rate_0.001647': 0.3365335144498499,
 '_dropout_0.581150_learning_rate_0.000310': -4.123225306815302,
 '_dropout_0.466913_learning_rate_0.003462': 0.30977173323640483,
 '_dropout_0.380887_learning_rate_0.005000': 0.5581399674502492,
 '_dropout_0.421808_learning_rate_0.005000': 0.42900747496414415,
 '_dropout_0.353065_learning_rate_0.000200': -1.2687765174239356,
 '_dropout_0.150000_learning_rate_0.005000': 0.6009713994268596,
 '_dropout_0.214420_learning_rate_0.005000': 0.549538962337399,
 '_dropout_0.393981_learning_rate_0.005000': 0.4722427643242765,
 '_dropout_0.247067_learning_rate_0.005000': 0.18921841526205152,
 '_dropout_0.184739_learning_rate_0.000200': 0.49346601531255485,
 '_dropout_0.153870_learning_rate_0.003664': 0.5786634529026986,
 '_dropout_0.203841_learning_rate_0.003891': 0.5100267062642383}

In [15]:
best_model.get_checkpoints()   # filename of best model saved in the folder "save_hyperparms"

['./result/Hyperparameter/model_dropout_0.150000_learning_rate_0.005000/ckpt-1']

### Load saved best model

In [16]:
best_model_checkpoints = best_model.get_checkpoints()[0]
best_model_folder = os.path.dirname(best_model_checkpoints)   # name of folder where best model is saved
best_model_folder

'./result/Hyperparameter/model_dropout_0.150000_learning_rate_0.005000'

In [18]:
# Initialize model with best hyperparameters
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', 
                  dropout=best_params['dropout'], learning_rate=best_params['learning_rate'],
                  model_dir=best_model_folder)

model.restore()

In [19]:
print("Training set score:", model.evaluate(train_dataset, metric_r2))
print("Validation set score:", model.evaluate(valid_dataset, metric_r2))
print("Test set score:", model.evaluate(test_dataset, metric_r2))

Training set score: {'r2_score': 0.7352561440217054}
Validation set score: {'r2_score': 0.6009713994268596}
Test set score: {'r2_score': 0.6062390140581315}


# Early Stopping

In [9]:
# Learning Rate Schedules (we don't use a fixed value for learning rate
# The learning rate starts as initial_rate. Every decay_steps training steps, it is multiplied by decay_rate.
# https://deepchem.readthedocs.io/en/latest/api_reference/models.html#deepchem.models.optimizers.ExponentialDecay
initial_rate = 0.001
decay_rate = 0.9
decay_steps = 200
learning_rate = dc.models.optimizers.ExponentialDecay(initial_rate, decay_rate, decay_steps)

#### We set "tensorboard=True" to log metric of validation dataset during training. 

In [23]:
del model

model = dc.models.GraphConvModel(n_tasks=n_tasks,              # No. of tasks 
                                 graph_conv_layers=[64],       # Width of channels for the Graph Convolution Layers
                                 dense_layer_size=128,         # Width of channels for Atom Level Dense Layer before GraphPool
                                 dropout=0.2,                  # Dropout probablity to use for each layer. The length of this list should equal len(graph_conv_layers)+1 (one value for each convolution layer, and one for the dense layer). Alternatively this may be a single value instead of a list, in which case the same value is used for every layer.
                                 mode='regression',            # Either “classification” or “regression”
                                 batch_size=100,               # Batch size for training and evaluating
                                 model_dir=model_dir,          # Directory on disk where the model will be stored. If this is None, a temporary directory is created.
                                 learning_rate=learning_rate,
                                 tensorboard=True              # log progress to TensorBoard during training                                 
                                 )

In [24]:
# Define "callback" function for early stopping
# It computes the validation set's metric every "interval" training steps (not epoch). 
interval = 100
callback = dc.models.ValidationCallback(valid_dataset, interval, metric_r2)

In [25]:
model.fit(train_dataset, nb_epoch=100, callbacks=callback)

/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_3/Reshape_14:0", shape=(354,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_3/Reshape_13:0", shape=(354, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_3/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/anaconda3/envs/GNN/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model_3/graph_pool_3/Reshape_17:0", shape=(1214,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_k

Step 100 validation: r2_score=0.413665
Step 200 validation: r2_score=0.398329
Step 300 validation: r2_score=0.514176
Step 400 validation: r2_score=0.541485
Step 500 validation: r2_score=0.602379
Step 600 validation: r2_score=0.634528
Step 700 validation: r2_score=0.609275
Step 800 validation: r2_score=0.61514


0.4439974212646484

In [26]:
print('training set score:', model.evaluate(train_dataset, metric_r2))
print('validation set score:', model.evaluate(valid_dataset, metric_r2))
print('test set score:', model.evaluate(test_dataset, metric_r2))

training set score: {'r2_score': 0.7951934295059822}
validation set score: {'r2_score': 0.615140117595991}
test set score: {'r2_score': 0.6539251579008657}


### Tensorboard

In order to see tensorboard, open Terminal. Go to the folder where you have saved this notebook by "cd" command. Then run the following command:

tensorboard --logdir=result

In the above command, "result" is the name of folder that model is saved.It is the value of parameter "model_dir". We set parameter "model_dir" when we initialized the model. 
Remember not to use '' marks in the above command for folder name. 

If you change the name of folder (set in the parameter "model_dir"), update the name of folder in the above command. 

This will launch the tensorboard web server on your local computer on port 6006. Open a new tab in your browser, and go to http://localhost:6006 in your web browser to look through tensorboard's UI.